In [1]:
import tensorflow as tf
from tensorflow.keras.regularizers import l2
import tensorflow_datasets as tfds
import numpy
from tensorflow.python.ops.gen_dataset_ops import shuffle_dataset
from sklearn.model_selection import train_test_split

dataset, info = tfds.load("movielens/100k-ratings", split=["train"], with_info=True)

data_set = dataset[0]

def preprocess_data(dataset):

    user_id = tf.strings.to_number(dataset["user_id"], out_type=tf.int32)
    movie_id = tf.strings.to_number(dataset["movie_id"], out_type=tf.int32)

    #casting user rating into 0 and 1 (0 means didn't rate this movie, 1 means did rate this movie)

    did_rate = tf.cast(dataset["user_rating"] > 0, tf.int64)

    features = {
        "user_id": user_id,
        "movie_id": movie_id,
        "user_rating":  did_rate
    }

    return features

data_set = data_set.map(preprocess_data)

print(len(data_set))

num_client = 5
userS_dataset = []

dataset_size = len(data_set)

for i in range(num_client):

    seed = numpy.random.seed(i)
    data_set = data_set.shuffle(buffer_size=dataset_size, seed= seed)
    userS_dataset.append(data_set)

data_set_1 =  data_set.shuffle(buffer_size=dataset_size, seed=42)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-ratings/incomplete.JDTYZL_0.1.1/movielens-train.tfrecord*..…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-ratings/0.1.1. Subsequent calls will reuse this data.
100000


In [2]:
#size of training data and testing data
train_size = int(dataset_size * 0.8)
test_size = dataset_size - train_size

#extract training data and testing data
train_data_1 = data_set_1.take(int(train_size))
test_data_1 = data_set_1.skip(int(train_size))

train_data = []
test_data = []

for i in range(num_client):

    train_data.append(userS_dataset[i].take(int(train_size)))
    test_data.append(userS_dataset[i].skip(int(train_size)))


In [3]:
userIDs = []
movieIDs = []
ratings = []

unique_user_movie_pair = set()
number = 0

for example in data_set:

    # Convert TensorFlow tensor to a NumPy value
    user_id = example["user_id"].numpy()
    movie_id = example["movie_id"].numpy()
    rating = example["user_rating"].numpy()

    # Add movie pair only if it is uniuqe
    if (user_id, movie_id) not in unique_user_movie_pair:
        userIDs.append(user_id)
        movieIDs.append(movie_id)
        ratings.append(rating)
        unique_user_movie_pair.add((user_id, movie_id))
        number += 1

print(f"Number of unique movie pairs: {number}")

# Convert lists to NumPy arrays for later processing
userIDs = numpy.array(userIDs)
movieIDs = numpy.array(movieIDs)
ratings = numpy.array(ratings)

movieIDs = numpy.unique(movieIDs)
userIDs = numpy.unique(userIDs)

#size of user and movie in training data
num_user = max(userIDs)
num_movie = max(movieIDs)

print(f"Number of unique users: {num_user}")
print(f"Number of unique movies: {num_movie}")


Number of unique movie pairs: 100000
Number of unique users: 943
Number of unique movies: 1682


In [4]:
triplets = list(zip(userIDs, movieIDs, ratings))

train_triplets, test_triplets = train_test_split(triplets, test_size=0.2, random_state=42)
train_matrix = numpy.zeros((num_user, num_movie), dtype=numpy.int32)
test_matrix = numpy.zeros((num_user, num_movie), dtype=numpy.int32)

def integrate_feature_into_matrix(userIDs, movieIDs, ratings, num_user, num_movie, matrix):

    # Create a 2D matrix filled with zeros
    # Populate the matrix
    for userID, movieID, rating in zip(userIDs, movieIDs, ratings):
        matrix[int(userID), int(movieID)] = rating

    return matrix

for user_id, movie_id, rating in train_triplets:
    train_matrix[user_id-1, movie_id-1] = rating

for user_id, movie_id, rating in test_triplets:
    test_matrix[user_id-1, movie_id-1] = rating

print(train_matrix.shape)
print(test_matrix.shape)
train_matrix = train_matrix.reshape((1, num_user, num_movie, 1))

print(train_matrix.shape)
print(test_matrix.shape)

(943, 1682)
(943, 1682)
(1, 943, 1682, 1)
(943, 1682)


In [24]:
def build_cnn_model(num_user, num_movie):

    regularization = l2(0.01)

    input_matrix = tf.keras.layers.Input(shape=(num_user, num_movie, 1), name="user_id")

    # First Convolution Block
    cnn_layer = tf.keras.layers.Conv2D(128, kernel_size=(6, 6),strides = (1,1), kernel_regularizer=regularization)(input_matrix)
    cnn_layer = tf.keras.layers.BatchNormalization()(cnn_layer)
    cnn_layer = tf.keras.layers.Activation('relu')(cnn_layer)
    cnn_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides = (1,1),)(cnn_layer)
    cnn_layer = tf.keras.layers.Dropout(0.1)(cnn_layer)

    # Second Convolution Block
    cnn_layer = tf.keras.layers.Conv2D(64, kernel_size=(4, 4), strides = (1,1), kernel_regularizer=regularization)(cnn_layer)
    cnn_layer = tf.keras.layers.BatchNormalization()(cnn_layer)
    cnn_layer = tf.keras.layers.Activation('relu')(cnn_layer)
    cnn_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides = (2,2),)(cnn_layer)
    cnn_layer = tf.keras.layers.Dropout(0.1)(cnn_layer)

    # Third Convolution Block
    cnn_layer = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), strides = (1,1),  kernel_regularizer=regularization)(cnn_layer)
    cnn_layer = tf.keras.layers.BatchNormalization()(cnn_layer)
    cnn_layer = tf.keras.layers.Activation('relu')(cnn_layer)
    cnn_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides = (3,3),)(cnn_layer)
    cnn_layer = tf.keras.layers.Dropout(0.1)(cnn_layer)

    cnn_layer = tf.keras.layers.Flatten()(cnn_layer)

    # Dense Layer
    dense_layer = tf.keras.layers.Dense(64, activation='relu')(cnn_layer)
    dense_layer = tf.keras.layers.Dropout(0.1)(dense_layer)

    dense_layer = tf.keras.layers.Dense(64, activation='relu')(dense_layer)
    dense_layer = tf.keras.layers.Dropout(0.1)(dense_layer)

    # Output Layer
    output = tf.keras.layers.Dense(num_movie, activation='sigmoid', name="movie_scores")(dense_layer)

    # Build and Compile the Model
    model = tf.keras.models.Model(inputs=input_matrix, outputs=output, name="MoviePopularityModel")

    model.compile(
        optimizer=tf.keras.optimizers.AdamW(learning_rate=0.001),
        loss= "mse",  # For regression tasks
        metrics=['accuracy']  # Mean Absolute Error for monitoring
    )

    return model

# Build the model
model = build_cnn_model(num_user, num_movie)

#target_train_matrix = train_matrix.reshape((train_matrix, num_movie))

# Train the model
model.fit(
    train_matrix,
    train_matrix,  # For self-supervised learning
    epochs=25,
)


Epoch 1/25


ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/tornado/ioloop.py", line 699, in <lambda>

  File "/usr/local/lib/python3.11/dist-packages/tornad

In [22]:
test_matrix_reshaped = test_matrix.reshape((1, num_user, num_movie, 1))
# Predict movie scores
predictions = model.predict(test_matrix_reshaped)

# Aggregate scores across users
movie_scores = predictions.mean(axis=0)

# Get top 50 movie indices
# Top 50 in descending ordere
movie_score = numpy.argsort(movie_scores)[:][::-1]

print("Predicted Movies Ranking(by index):")
for rank, movie_idx in enumerate(movie_score, 1):
    print(f"{rank}: Movie Index {movie_idx}, Score {movie_scores[movie_idx] } ")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 670ms/step
Predicted Movies Ranking(by index):
1: Movie Index 1263, Score 0.4988913834095001 
2: Movie Index 309, Score 0.4988466203212738 
3: Movie Index 1237, Score 0.4988413453102112 
4: Movie Index 1258, Score 0.498817503452301 
5: Movie Index 273, Score 0.4988076090812683 
6: Movie Index 46, Score 0.4987960457801819 
7: Movie Index 99, Score 0.4987756311893463 
8: Movie Index 1242, Score 0.4987702965736389 
9: Movie Index 345, Score 0.4987565875053406 
10: Movie Index 1551, Score 0.49874016642570496 
11: Movie Index 1157, Score 0.49872973561286926 
12: Movie Index 1241, Score 0.4987245500087738 
13: Movie Index 339, Score 0.4987221360206604 
14: Movie Index 739, Score 0.49871644377708435 
15: Movie Index 1334, Score 0.49871382117271423 
16: Movie Index 872, Score 0.4987093210220337 
17: Movie Index 886, Score 0.49870747327804565 
18: Movie Index 1233, Score 0.49870091676712036 
19: Movie Index 411, Score 0.4986911714076996 
20: Movie Index 1571, Score 0

In [7]:
# loss, mae = model.evaluate(train_matrix, test_matrix)  # For self-supervised learning, use test_matrix as both inputs and targets
# print(f"Test Loss: {loss}")
# print(f"Test MAE: {mae}")


In [23]:
def calculate_caching_hit_rate(predicted_scores, test_matrix, cache_size_array):

    """
    Calculate the caching hit rate.

    :param predicted_scores: Array of predicted scores for all movies (shape: num_movies).
    :param test_data: Test dataset containing actual movie requests (e.g., movieIDs).
    :param cache_size: Number of movies to cache (top-k based on predicted scores).
    :return: Hit rate as a float.
    """

    predicted_scores = predicted_scores.flatten()

    num_user, num_movie = test_matrix.shape

    for cache_size in cache_size_array:

      # Get top-k movies based on predicted scores
      top_k_movies = numpy.argsort(predicted_scores)[-cache_size:][::-1]

      #Extract actual requested movie IDs from test_matrix
      requested_movie_ids = set()
      hits = 0

      for user_id in range(num_user):

          requested_movies = numpy.where(test_matrix[user_id] == 1)[0]
          requested_movie_ids.update(requested_movies)

      # Calculate hit rate
      for movie_id in requested_movie_ids:
          if movie_id in top_k_movies:
              hits += 1

      total_unique_movies = len(requested_movie_ids)
      print(f"Total unique movies: {total_unique_movies}, total hit: {hits}")

      hit_rate = hits / cache_size
      print(f"Hit Rate for cache size {cache_size}: {hit_rate:.2%}")
      print("")

    return hit_rate

# Example usage
cache_size_array = [50, 100, 150, 200, 250, 300]  # the number of cached top N movies
hit_rate = calculate_caching_hit_rate(predictions, test_matrix, cache_size_array)

Total unique movies: 189, total hit: 5
Hit Rate for cache size 50: 10.00%

Total unique movies: 189, total hit: 12
Hit Rate for cache size 100: 12.00%

Total unique movies: 189, total hit: 19
Hit Rate for cache size 150: 12.67%

Total unique movies: 189, total hit: 24
Hit Rate for cache size 200: 12.00%

Total unique movies: 189, total hit: 29
Hit Rate for cache size 250: 11.60%

Total unique movies: 189, total hit: 35
Hit Rate for cache size 300: 11.67%

